In [4]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q htt  https://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


# Read the data

In [49]:
df=spark.read.format("csv").option("header","true").option("inferschema","true").load("/content/walmart_stock.csv");
df.show()

+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|               Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+------------------+------------------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|         59.549999|         58.919998|             59.18| 6679300|51.616215000

# **How Spark infered the columns?**

In [50]:
df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



# **Column Names**

In [83]:
ms=df.columns;
print(ms)

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']


# **Select first 5 columns**

In [55]:
df.select(ms[0:5]).show()

+-------------------+------------------+------------------+------------------+------------------+
|               Date|              Open|              High|               Low|             Close|
+-------------------+------------------+------------------+------------------+------------------+
|2012-01-03 00:00:00|         59.970001|         61.060001|         59.869999|         60.330002|
|2012-01-04 00:00:00|60.209998999999996|         60.349998|         59.470001|59.709998999999996|
|2012-01-05 00:00:00|         59.349998|         59.619999|         58.369999|         59.419998|
|2012-01-06 00:00:00|         59.419998|         59.450001|         58.869999|              59.0|
|2012-01-09 00:00:00|         59.029999|         59.549999|         58.919998|             59.18|
|2012-01-10 00:00:00|             59.43|59.709998999999996|             58.98|59.040001000000004|
|2012-01-11 00:00:00|         59.060001|         59.529999|59.040001000000004|         59.400002|
|2012-01-12 00:00:00

# **Select first 5 rows**

In [56]:
df.show(5)

+-------------------+------------------+---------+---------+------------------+--------+------------------+
|               Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03 00:00:00|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04 00:00:00|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05 00:00:00|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06 00:00:00|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09 00:00:00|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+-------------------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [57]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

In [59]:
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



# **Create a new dataframe with a column called HV Ratio**


In [61]:
df2=df.withColumn('HV Ratio',df['High']/df['Volume']);
df2.select('HV Ratio').show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



# **What day had Peak High in Price?**

In [62]:
from pyspark.sql.functions import *;

In [66]:
df.filter(df['High']==((((df.select(max('High').alias('Maximum_High')).collect())[0]).asDict())['Maximum_High'])).select('Date').show()

+-------------------+
|               Date|
+-------------------+
|2015-01-13 00:00:00|
+-------------------+



# **What is the mean of the Close Column?**

In [84]:
df.select(avg('Close').alias('Average_Value')).show()

+-----------------+
|    Average_Value|
+-----------------+
|72.38844998012726|
+-----------------+



# **What is the max & min of the Volumne Column?**

In [69]:
df.select(max('Volume'),min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



# **How many days was the Close lower than 60 dollars?**

In [70]:
df.filter(df['Close']<60).count()

81

# **What percentage of the time was the High greater than 80 dollars?**

In [72]:
((df.filter(df['High']>80).count())/df.count())*100

9.141494435612083

# **What is the Pearson Correlation between High & Volume?**

In [75]:
df.select(corr('High','Volume')).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



# **What is the max High per year?**

In [77]:
df2=df.withColumn('Year',year('Date'));
df2.groupBy('Year').agg(max('High')).show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



# **What is the average Close for each Calender Month?**

In [81]:
df2=df.withColumn('montha',month('Date'));
df2.groupBy('montha').agg(avg('Close')).show()

+------+-----------------+
|montha|       avg(Close)|
+------+-----------------+
|    12|72.84792478301885|
|     1|71.44801958415842|
|     6| 72.4953774245283|
|     3|71.77794377570092|
|     5|72.30971688679247|
|     9|72.18411785294116|
|     4|72.97361900952382|
|     8|73.02981855454546|
|     7|74.43971943925233|
|    10|71.57854545454543|
|    11| 72.1110893069307|
|     2|  71.306804443299|
+------+-----------------+

